In [2]:
%serialconnect /dev/tty.usbmodem5A7A0440001

Found serial ports: /dev/cu.usbmodem21101, /dev/cu.usbmodem5A7A0440001, /dev/cu.Bluetooth-Incoming-Port, /dev/cu.debug-console 
Connecting to --port=/dev/cu.usbmodem21101 --baud=115200 
Ready.


In [ ]:
# RSCM171100k Pressure sensor using ADC input

from machine import ADC, Pin
import time

# esp32 can't take 4.5 V directly. Thus use a voltage divider with R1 = 2k and R2 = 4.7k
adc = ADC(Pin(4))
adc.atten(ADC.ATTN_11DB)

R1 = 2
R2 = 4.7
DIVIDER_GAIN = (R1 + R2) / R2

def stream_voltage(tm):

    


    start_time = time.ticks_us()
    i = 0
    voltage_accum = 0
    while True:
        reading = adc.read_u16() / 65535
        voltage = reading * 3.3 * DIVIDER_GAIN

        voltage_accum = voltage_accum + voltage if i > 0 else voltage
        i = i + 1

        current_time = time.ticks_us()
        if time.ticks_diff(current_time, start_time) > 1e6:
            voltage_avg = voltage_accum / i
            print("Average Voltage: {:.4f} V".format(voltage_avg))
            break

    last_time = time.ticks_us()
    a = 0.1
    fps = None

    while True:
        reading = adc.read_u16() / 65535
        voltage = reading * 3.3 * DIVIDER_GAIN

        pressure = (voltage - voltage_avg) * 2e4 /4
        

        current_time = time.ticks_us()
        dt = time.ticks_diff(current_time, last_time)
        last_time = current_time
        fps_new = 1e6 / dt
        fps = 0 if fps is None else a * fps_new + (1 - a) * fps

        #print("ADC Reading: {:.4f}, Voltage: {:.4f} V".format(reading, voltage))
        #print([voltage, fps])
        print([pressure])

        time.sleep_ms(tm)

In [43]:
stream_voltage(0)

Average Voltage: 0.4568 V
[-15.29068]
[-26.7759]
[7.679611]
[-15.29068]
[-3.805608]
[7.679611]
[-3.805608]
[-3.805608]
[-3.805608]
[7.679611]
[-3.805608]
[-15.29068]
[-21.03344]
[-15.29068]
[-38.26112]
[-3.805608]
[-26.7759]
[-15.29068]
[-26.7759]
[-26.7759]
[-3.805608]
[-21.03344]
[-15.29068]
[1.936853]
[-26.7759]
[-3.805608]
[-3.805608]
[-3.805608]
[-3.805608]
[-15.29068]
[-3.805608]
[-32.51865]
[1.936853]
[-15.29068]
[-32.51865]
[30.6499]
[-3.805608]
[-26.7759]
[19.16483]
[36.39236]
[13.42207]
[-3.805608]
[-26.7759]
[1.936853]
[-3.805608]
[-15.29068]
[1.936853]
[-15.29068]
[-3.805608]
[-3.805608]
[-15.29068]
[-3.805608]
[-9.548367]
[-26.7759]
[-9.548367]
[-15.29068]
[-15.29068]
[7.679611]
[-3.805608]
[7.679611]
[30.6499]
[7.679611]
[-15.29068]
[-9.548367]
[7.679611]
[-15.29068]
[-49.74619]
[-26.7759]
[-26.7759]
[-3.805608]
[-15.29068]
[7.679611]
[-32.51865]
[-3.805608]
[-3.805608]
[7.679611]
[-15.29068]
[19.16483]
[-3.805608]
[7.679611]
[1.936853]
[-3.805608]
[-21.03344]
[-3.805608]

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "<stdin>", line 48, in stream_voltage
KeyboardInterrupt: 


In [ ]:
# MicroPython example for XGZP6897D pressure sensor (I2C)
# - Prints pressure (Pa) and temperature (°C) in real time

from machine import Pin, I2C
import time

# ========= USER SETTINGS (change these to match your board) =========
I2C_ID  = 0            # try 0 first; if scan() shows nothing, try 1
SCL_PIN = 9           # set to your SCL pin
SDA_PIN = 8           # set to your SDA pin
K = 128                # scale factor (Pa = ADC / K). Pick by range:
#   Range (kPa) → K:
#   1000–2000: 4     500–1000: 8    260–500: 16   131–260: 32
#    65–131: 64       32–65: 128     16–32: 256     8–16: 512
#     4–8: 1024        2–4: 2048       1–2: 4096     0.5–1: 8192
# Example: a 30–80 kPa sensor has span P=50 kPa → K=128

# ========= SENSOR CONSTANTS (from datasheet) =========
ADDR = 0x6D            # 7-bit I2C address
REG_CMD       = 0x30   # command register: [sleep_time:4][Sco:1][MeasCtrl:2]
REG_PRESS_MSB = 0x06   # pressure 24-bit: 0x06, 0x07, 0x08 (MSB..LSB)
REG_TEMP_MSB  = 0x09   # temp 16-bit:     0x09 (MSB), 0x0A (LSB)
REG_ASIC_CFG  = 0xA5   # config register: clear RAW_DATA_ON bit to get calibrated values

# ========= I2C SETUP =========
i2c = I2C(I2C_ID, scl=Pin(SCL_PIN), sda=Pin(SDA_PIN), freq=400_000)  # 400 kHz per datasheet
print("I2C scan:", [hex(d) for d in i2c.scan()])  # expect to see 0x6d

I2C scan: ['0x6d']


In [ ]:


def reg_read8(reg):
    return i2c.readfrom_mem(ADDR, reg, 1)[0]

def reg_write8(reg, val):
    i2c.writeto_mem(ADDR, reg, bytes([val & 0xFF]))

# ========= INIT (optional but recommended) =========
# Put device into "calibrated output" mode by clearing RAW_DATA_ON (bit1) in 0xA5.
# If your module already ships calibrated, this will be a no-op.
try:
    cfg = reg_read8(REG_ASIC_CFG)
    cfg &= 0b11111101          # clear bit1
    reg_write8(REG_ASIC_CFG, cfg)
except OSError:
    # Some variants may not expose 0xA5; safe to continue.
    pass

def start_combined_conversion():
    """
    Kick off one combined measurement (temperature then pressure):
    - Measurement_ctrl=0b10
    - Sco=1 (start once)
    - sleep_time=0
    That byte is 0x0A. Then poll until Sco goes back to 0.
    """
    reg_write8(REG_CMD, 0x0A)
    # Poll Sco bit (bit3) until it clears (conversion finished)
    while reg_read8(REG_CMD) & 0x08:
        time.sleep_ms(2)

def read_pressure_temperature(k=K):
    """
    Returns (pressure_pa, temperature_celsius).
    Sign handling:
      - Pressure is signed 24-bit (two’s complement)
      - Temperature is signed 16-bit (two’s complement), LSB = 1/256 °C
    """
    start_combined_conversion()

    # Read pressure 24-bit
    p3 = i2c.readfrom_mem(ADDR, REG_PRESS_MSB, 3)
    press_adc = (p3[0] << 16) | (p3[1] << 8) | p3[2]
    if press_adc & (1 << 23):           # negative value?
        press_adc -= (1 << 24)          # sign-extend 24-bit

    # Read temperature 16-bit
    t2 = i2c.readfrom_mem(ADDR, REG_TEMP_MSB, 2)
    temp_adc = (t2[0] << 8) | t2[1]
    if temp_adc & (1 << 15):            # negative value?
        temp_adc -= (1 << 16)           # sign-extend 16-bit

    pressure_pa = press_adc / float(k)  # per datasheet’s ADC→Pa formula
    temperature_c = temp_adc / 256.0    # per datasheet (LSB = 1/256 °C)

    return pressure_pa, temperature_c

# ========= SIMPLE "REAL-TIME" PRINT LOOP =========
# Tip: use Ctrl+C in the notebook cell to stop the loop.
while True:
    try:
        p, t = read_pressure_temperature(K)
        print("P = {:.1f} Pa,  T = {:.2f} °C".format(p, t))
    except OSError as e:
        # I2C read/write failed (e.g., loose wire). Keep going.
        print("I2C error:", e)
    time.sleep_ms(10)  # ~10 Hz update; adjust as you like
    


P = 147.9 Pa,  T = 28.96 °C
P = 150.0 Pa,  T = 28.96 °C
P = 145.5 Pa,  T = 28.96 °C
P = 142.2 Pa,  T = 28.96 °C
P = 145.3 Pa,  T = 28.96 °C
P = 143.1 Pa,  T = 28.96 °C
P = 136.0 Pa,  T = 28.98 °C
P = 147.3 Pa,  T = 28.96 °C
P = 152.5 Pa,  T = 28.97 °C
P = 143.9 Pa,  T = 28.97 °C
P = 147.5 Pa,  T = 28.97 °C
P = 143.9 Pa,  T = 28.99 °C
P = 144.4 Pa,  T = 28.98 °C
P = 147.1 Pa,  T = 28.97 °C
P = 144.6 Pa,  T = 28.98 °C
P = 145.8 Pa,  T = 28.97 °C
P = 138.6 Pa,  T = 28.97 °C
P = 146.3 Pa,  T = 28.98 °C
P = 147.8 Pa,  T = 28.98 °C
P = 147.9 Pa,  T = 28.98 °C
P = 147.7 Pa,  T = 28.98 °C
P = 144.1 Pa,  T = 28.97 °C
P = 144.2 Pa,  T = 28.98 °C
P = 140.0 Pa,  T = 28.99 °C
P = 137.6 Pa,  T = 28.98 °C
P = 149.1 Pa,  T = 28.99 °C
P = 146.9 Pa,  T = 28.98 °C
P = 144.3 Pa,  T = 28.99 °C
P = 141.3 Pa,  T = 28.98 °C
P = 153.1 Pa,  T = 28.98 °C
P = 148.0 Pa,  T = 28.99 °C
P = 137.2 Pa,  T = 28.98 °C
P = 144.9 Pa,  T = 28.98 °C
P = 133.7 Pa,  T = 28.98 °C
P = 151.1 Pa,  T = 28.98 °C
P = 153.9 Pa,  T = 2

Traceback (most recent call last):
  File "<stdin>", line 63, in <module>
  File "<stdin>", line 40, in read_pressure_temperature
  File "<stdin>", line 31, in start_combined_conversion
KeyboardInterrupt: 


In [ ]:

start_time = time.ticks_us()
i = 0
p_i2c_accum = 0
voltage_accum = 0
while True:
    p_i2c, t_i2c = read_pressure_temperature(K)
    p_i2c_accum = p_i2c_accum + p_i2c if i > 0 else p_i2c
    i = i + 1

    reading = adc.read_u16() / 65535
    voltage = reading * 3.3 * DIVIDER_GAIN
    voltage_accum = voltage_accum + voltage if i > 0 else voltage

    current_time = time.ticks_us()
    time.sleep_ms(10)
    if time.ticks_diff(current_time, start_time) > 1e6:
        p_i2c_avg = p_i2c_accum / i
        voltage_avg = voltage_accum / i

        print("Average Voltage: {:.4f} V".format(voltage_avg))
        print("Average I2C Pressure: {:.1f} Pa".format(p_i2c_avg))
        break
    
while True:
    p_i2c, t_i2c = read_pressure_temperature(K)
    p_i2c = p_i2c - p_i2c_avg

    reading = adc.read_u16() / 65535
    voltage = reading * 3.3 * DIVIDER_GAIN
    p_adc = (voltage - voltage_avg) * 2e4 /4

    print([p_adc, p_i2c])
    #print((p_adc-p_i2c)/p_i2c)
    time.sleep_ms(10)
    

    

Average Voltage: 0.4556 V
Average I2C Pressure: 143.2 Pa
[13.62786, 16.66318]
[-15.08519, 16.53818]
[-9.342432, 2.23349]
[2.142638, 4.944427]
[2.142638, 5.249115]
[-20.82765, -0.6024475]
[30.85539, -5.61026]
[7.885098, 7.530365]
[-3.600121, -4.118073]
[30.85539, -0.8368225]
[-9.342432, 10.10068]
[-20.82765, 7.32724]
[-3.600121, 3.053802]
[-9.342432, 9.48349]
[13.62786, 10.67099]
[13.62786, 0.8350525]
[36.59814, 15.35849]
[2.142638, 0.92099]
[-3.600121, -5.625885]
[30.85539, 7.280365]
[2.142638, 12.1788]
[25.11308, -0.3524475]
[-15.08519, 7.32724]
[36.59814, 4.79599]
[-26.57041, 5.178802]
[30.85539, 2.749115]
[-26.57041, 10.04599]
[7.885098, 8.63974]
[30.85539, 11.79599]
[13.62786, 8.444427]
[2.142638, -0.875885]
[-26.57041, 4.491302]
[-9.342432, 4.561615]
[48.08337, 9.444427]
[30.85539, 7.592865]
[42.34061, 6.45224]
[7.885098, 7.79599]
[30.85539, 3.405365]
[-20.82765, 3.57724]
[7.885098, 13.0538]
[2.142638, 2.835052]
[-9.342432, 5.788177]
[-9.342432, -1.711823]
[-3.600121, 4.491302]
[3

Traceback (most recent call last):
  File "<stdin>", line 25, in <module>
  File "<stdin>", line 40, in read_pressure_temperature
  File "<stdin>", line 31, in start_combined_conversion
KeyboardInterrupt: 
